In [ ]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, count, col, desc, format_number

base_path = r'G:\.shortcut-targets-by-id\1eDKYURr-Qm222Ul6PoZLc4b99cuwHTC3\Cường\Đại học\DS200\Lab 1'

# Khởi tạo SparkSession
spark = SparkSession.builder \
    .appName("DS200 Lab1 - Bai 1") \
    .getOrCreate()

try:
    movies_df = spark.read.format("csv") \
        .option("header", "false") \
        .option("inferSchema", "true") \
        .load(os.path.join(base_path, "movies.cleaned.txt")) \
        .toDF("MovieID", "Title", "Genres")

    ratings_temp_df = spark.read.format("csv") \
        .option("header", "false") \
        .load(os.path.join(base_path, "ratings.cleaned.txt")) \
        .toDF("UserID_str", "MovieID_str", "Rating_str", "Timestamp_str")

    ratings_df = ratings_temp_df.select(
        col("UserID_str").cast("int").alias("UserID"),
        col("MovieID_str").cast("int").alias("MovieID"),
        col("Rating_str").cast("float").alias("Rating"),
        col("Timestamp_str").cast("long").alias("Timestamp")
    )

    print("\nSchema của movies_df:")
    movies_df.printSchema()
    print("\nSchema của ratings_df (sau khi đã chuyển đổi kiểu):")
    ratings_df.printSchema()

    print(f"Số lượng dòng trong ratings_df: {ratings_df.count()}")
    ratings_df.show(5)

except Exception as e:
    print(f"Lỗi khi đọc file: {e}")
    spark.stop()
    exit()

Tổng số phim: 50
Tổng số rating: 184
Mẫu phim: [(1001, ('The Godfather (1972)', 'Crime|Drama'))]
Mẫu rating: [(7, 1020, 4.5, 1577836800)]


# Tính điểm đánh giá trung bình và tổng số lượt đánh giá

In [2]:
movie_stats_df = ratings_df.groupBy("MovieID").agg(
    avg("Rating").alias("AverageRating"),
    count("Rating").alias("TotalRatings")
)

In [3]:
final_df = movie_stats_df.join(movies_df, "MovieID")

# Hiển thị kết quả theo định dạng: MovieTitle AverageRating: xx (TotalRatings: xx)
final_df.select(
    col("Title"),
    format_number(col("AverageRating"), 2).alias("AverageRating"), 
    col("TotalRatings")
).orderBy(col("Title")).show(200, truncate=False) 

+---------------------------------------------------------+-------------+------------+
|Title                                                    |AverageRating|TotalRatings|
+---------------------------------------------------------+-------------+------------+
| American Beauty (1999)                                  |3.75         |2           |
| Avatar (2009)                                           |4.75         |2           |
| Back to the Future (1985)                               |4.00         |2           |
| Birdman (2014)                                          |3.00         |2           |
| Braveheart (1995)                                       |4.25         |2           |
| Coco (2017)                                             |4.25         |2           |
| Dunkirk (2017)                                          |3.50         |2           |
| Fight Club (1999)                                       |4.25         |2           |
| Forrest Gump (1994)                      

# Tìm phim có điểm trung bình cao nhất

In [4]:
filtered_for_max_df = final_df.filter(col("TotalRatings") >= 5)

# Sắp xếp theo điểm trung bình giảm dần để tìm phim cao nhất
highest_rated_movie = filtered_for_max_df.orderBy(desc("AverageRating")).first()

if highest_rated_movie:
    title = highest_rated_movie['Title']
    avg_rating = highest_rated_movie['AverageRating']

    print(f"'{title}' is the highest rated movie with an average rating of "
          f"{avg_rating:.2f} among movies with at least 5 ratings.")
else:
    print("\nKhông tìm thấy phim nào có tối thiểu 5 lượt đánh giá.")


Không tìm thấy phim nào có tối thiểu 5 lượt đánh giá.


In [5]:
spark.stop()
